# Data Sharing with Encryption - Delentture side

## Recommended 

Create conda enviroment to avoid conflicts with packages.

## Data Analysis

In [1]:
import pandas as pd
import os

pd.options.display.max_colwidth = 200

# get current directory
path = os.getcwd()
parent = os.path.dirname(path)

DIR_DATA = parent+ '/data/'

In [2]:
df_columns = pd.read_csv(DIR_DATA+'columns_description.csv')
df_loans = pd.read_csv(DIR_DATA+'infringement_dataset.csv')

In [3]:
df_columns.head()

,New name,Description
0,address,Client's address
1,age,Client's age in days at the time of application
2,annual_income,Income of the client
3,appendix_a,Did client provide document 2
4,appendix_b,Did client provide document 3


In [4]:
df_loans.head()

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.000000,9251.775,179055.00,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.000000,56553.990,435436.50,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.000000,5357.250,24282.00,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,100006.0,272.444444,23651.175,272203.26,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,100007.0,1222.833333,12278.805,150530.25,166638.75,6.0,0.0,0.0,0.0,6.0


In [5]:
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 69 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   loan_id                   307511 non-null  int64  
 1   infringed                 307511 non-null  int64  
 2   contract_type             307511 non-null  object 
 3   gender                    307511 non-null  object 
 4   has_own_car               307511 non-null  object 
 5   has_own_realty            307511 non-null  object 
 6   num_children              307511 non-null  int64  
 7   annual_income             307511 non-null  float64
 8   credit_amount             307511 non-null  float64
 9   credit_annuity            307499 non-null  float64
 10  goods_valuation           307233 non-null  float64
 11  income_type               307511 non-null  object 
 12  education                 307511 non-null  object 
 13  family_status             307511 non-null  o

### Data Pre-Processing

In [6]:
def check_nans(df,cols = None):
    if cols is None:
        cols = df.columns
    for col in cols:
        print(f'{col} - {df[col].isnull().sum()}')

def check_unique(df, cols = None):
    if cols is None:
        cols = df.columns
    for col in cols:
        print(f'{col} - {df[col].unique()}\n')  


df_loans_processed = df_loans.copy()
df_loans_processed = df_loans_processed.drop(columns = ['first_name','last_name','address'])
df_loans_processed.shape

(307511, 66)

#### Cleaning
Handle Missing/Wrong Values by checking the importance and number of rows with null or invalid values.

In [7]:
cols_aux = ['region_rating','mobilephone_reachable','SK_ID_CURR','car_age','family_status','gender','goods_valuation']
cond_regex = df_columns['New name'].isin(cols_aux)

df_columns[cond_regex]

,New name,Description
23,car_age,Age of client's car
29,family_status,Family status of the client
31,gender,Gender of the client
32,goods_valuation,For consumer loans it is the price of the goods for which the loan is given
40,mobilephone_reachable,"Was mobile phone reachable (1=YES, 0=NO)"
63,region_rating,"Our rating of the region where client lives (1,2,3)"


In [8]:
df_loans_processed[cols_aux].head()

,region_rating,mobilephone_reachable,SK_ID_CURR,car_age,family_status,gender,goods_valuation
0,2,1,100002.0,NaN,Single / not married,M,351000.0
1,1,1,100003.0,NaN,Married,F,1129500.0
2,2,1,100004.0,26.0,Single / not married,M,135000.0
3,2,1,100006.0,NaN,Civil marriage,F,297000.0
4,2,1,100007.0,NaN,Single / not married,M,513000.0


In [9]:
check_nans(df_loans_processed,cols_aux)
df_loans_processed = df_loans_processed.drop(columns = ['SK_ID_CURR','car_age'])
df_loans_processed.shape[1]

region_rating - 0
mobilephone_reachable - 0
SK_ID_CURR - 16454
car_age - 202929
family_status - 0
gender - 0
goods_valuation - 278


64

Removed the following columns:
- **SK_ID_CURR** - The main reason is that this label does not have a description on the dataset where it's supposed to be, so we cannot use this data since we don´t know what represents. Besides, it has some missing values associated with
- **car_age** - 202929 entries have this value missing and also its importance is small

In [10]:
cols_unique = ['family_status','gender','region_rating','mobilephone_reachable']
check_unique(df_loans_processed,cols_unique)
print(f'Rows of dataset: {df_loans_processed.shape[0]}\n')
df_loans_processed = df_loans_processed[df_loans_processed.gender != 'XNA']
check_unique(df_loans_processed,['gender'])
print(f'Rows of dataset: {df_loans_processed.shape[0]}')

family_status - ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']

gender - ['M' 'F' 'XNA']

region_rating - [2 1 3]

mobilephone_reachable - [1 0]

Rows of dataset: 307511

gender - ['M' 'F']

Rows of dataset: 307507


There were only 4 rows with a invalid value in the column gender. So removing those entries won't do much harm.

- Labels starting with **'provided_'**

In [11]:
regex_provided = 'provided_.*'
cond_regex = df_columns['New name'].str.match(regex_provided)
cols_aux = df_columns[cond_regex]['New name'].tolist()
df_columns[cond_regex]

,New name,Description
59,provided_email,"Did client provide email (1=YES, 0=NO)"
60,provided_homephone,"Did client provide home phone (1=YES, 0=NO)"
61,provided_mobilephone,"Did client provide mobile phone (1=YES, 0=NO)"
62,provided_workphone,"Did client provide work phone (1=YES, 0=NO)"


In [12]:
check_unique(df_loans_processed,cols_aux)

provided_email - [0 1]

provided_homephone - [0 1]

provided_mobilephone - [1 0]

provided_workphone - [1 0]



- Labels starting with **_type**

In [13]:
regex_type = '.*_type'
cond_regex = df_columns['New name'].str.match(regex_type)
cols_aux = df_columns[cond_regex]['New name'].tolist()
df_columns[cond_regex]

,New name,Description
24,contract_type,Identification if loan is cash or revolving
35,housing_type,"What is the housing situation of the client (renting, living with parents, ...)"
36,income_type,"Clients income type (businessman, working, maternity leave,…)"
49,occupation_type,What kind of occupation does the client have
50,organization_type,Type of organization where client works


In [14]:
check_unique(df_loans_processed,cols_aux)
check_nans(df_loans_processed,cols_aux)
print(f'Shape of datatset: {df_loans_processed.shape}')

contract_type - ['Cash loans' 'Revolving loans']

housing_type - ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']

income_type - ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']

occupation_type - ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']

organization_type - ['Business Entity Type 3' 'School' 'Government' 'Religion' 'Other' 'XNA'
 'Electricity' 'Medicine' 'Business Entity Type 2' 'Self-employed'
 'Transport: type 2' 'Construction' 'Housing' 'Kindergarten'
 'Trade: type 7' 'Industry: type 11' 'Military' 'Services'
 'Security Ministries' 'Transport: type 4' 'Industry: type 1' 'Emergency'
 'Security' 'Trad

In [15]:
df_loans_processed = df_loans_processed[df_loans_processed['organization_type'] != 'XNA']

regex_industry = 'Industry:.*'
df_loans_processed.loc[df_loans_processed['organization_type'].str.match(regex_industry),'organization_type'] = 'Industry'

regex_trade = 'Trade:.*'
df_loans_processed.loc[df_loans_processed['organization_type'].str.match(regex_trade),'organization_type'] = 'Trade'

regex_transport = 'Transport:.*'
df_loans_processed.loc[df_loans_processed['organization_type'].str.match(regex_transport),'organization_type'] = 'Transport'

regex_business= 'Business Entity.*'
df_loans_processed.loc[df_loans_processed['organization_type'].str.match(regex_business),'organization_type'] = 'Business Entity'

check_unique(df_loans_processed,['organization_type'])
df_loans_processed = df_loans_processed.drop(columns = ['occupation_type'])
print(f'Shape of datatset: {df_loans_processed.shape}')

organization_type - ['Business Entity' 'School' 'Government' 'Religion' 'Other' 'Electricity'
 'Medicine' 'Self-employed' 'Transport' 'Construction' 'Housing'
 'Kindergarten' 'Trade' 'Industry' 'Military' 'Services'
 'Security Ministries' 'Emergency' 'Security' 'University' 'Police'
 'Postal' 'Agriculture' 'Restaurant' 'Culture' 'Hotel' 'Bank' 'Insurance'
 'Mobile' 'Legal Services' 'Advertising' 'Cleaning' 'Telecom' 'Realtor']

Shape of datatset: (252133, 63)


- Since the organization_type label description doesen´t specify the differences between the types in a specific category, the size of unique values was reduced
- Removed rows from the column **organization_type** where it had the value **XNA**
- Removed **occupation_type** column due to the number of missing values. Also it has very small importance, since the dataset is about loans the column organization_type gives a more meaningful information about the client and its more easily analyzed.

In [16]:
regex_has = 'has.*'
df_columns[df_columns['New name'].str.match(regex_has)]

,New name,Description
33,has_own_car,Flag if the client owns a car
34,has_own_realty,Flag if client owns a house or flat


In [17]:
df_has = df_loans_processed.filter(regex= (regex_has))
check_unique(df_has)

has_own_car - ['N' 'Y']

has_own_realty - ['Y' 'N']



In [18]:
regex_score = 'score.*'
df_columns[df_columns['New name'].str.match(regex_score)]

,New name,Description
64,score_ext_1,Normalized score from external data source
65,score_ext_2,Normalized score from external data source
66,score_ext_3,Normalized score from external data source


In [19]:
df_score = df_loans_processed.filter(regex= (regex_score))
df_score.head()

,score_ext_1,score_ext_2,score_ext_3
0,0.083037,0.262949,0.139376
1,0.311267,0.622246,NaN
2,NaN,0.555912,0.729567
3,NaN,0.650442,NaN
4,NaN,0.322738,NaN


In [20]:
check_nans(df_score)
df_loans_processed = df_loans_processed.drop(columns = df_score.columns.tolist())
print(f'Num of Cols: {df_loans_processed.shape[1]}')

score_ext_1 - 130859
score_ext_2 - 504
score_ext_3 - 49896
Num of Cols: 60


- Removed columns **score_ext_1**, **score_ext_2** and **score_ext_3** due to missing values and low significal importance. And also the description of the label is not clear enough

In [21]:
regex_appendix = 'appendix.*'
df_columns[df_columns['New name'].str.match(regex_appendix)].head()

,New name,Description
3,appendix_a,Did client provide document 2
4,appendix_b,Did client provide document 3
5,appendix_c,Did client provide document 4
6,appendix_d,Did client provide document 5
7,appendix_e,Did client provide document 6


In [22]:
df_appendix = df_loans_processed.filter(regex= (regex_appendix))
check_unique(df_appendix)

appendix_a - [0 1]

appendix_b - [1 0]

appendix_c - [0 1]

appendix_d - [0 1]

appendix_e - [0 1]

appendix_f - [0 1]

appendix_g - [0 1]

appendix_h - [0 1]

appendix_i - [0 1]

appendix_j - [0 1]

appendix_k - [0 1]

appendix_l - [0 1]

appendix_m - [0 1]

appendix_n - [0 1]

appendix_o - [0 1]

appendix_p - [0 1]

appendix_q - [0 1]

appendix_r - [0 1]

appendix_s - [0 1]

appendix_t - [0 1]



In [23]:
regex_num = 'num_.*'
df_columns[df_columns['New name'].str.match(regex_num)]

,New name,Description
41,num_children,Number of children the client has
42,num_family_members,How many family members does client have
43,num_req_bureau_day,Number of enquiries to Credit Bureau about the client one day before application (excluding one hour before application)
44,num_req_bureau_hour,Number of enquiries to Credit Bureau about the client one hour before application
45,num_req_bureau_month,Number of enquiries to Credit Bureau about the client one month before application (excluding one week before application)
46,num_req_bureau_qrt,Number of enquiries to Credit Bureau about the client 3 month before application (excluding one month before application)
47,num_req_bureau_week,Number of enquiries to Credit Bureau about the client one week before application (excluding one day before application)
48,num_req_bureau_year,Number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application)


In [24]:
df_loans_processed = df_loans_processed.drop(columns = ['num_req_bureau_hour'])

df_num = df_loans_processed.filter(regex= (regex_num))
check_unique(df_num)
check_nans(df_num)

print(f'Num of Cols: {df_loans_processed.shape[1]}')

num_children - [ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]

num_family_members - [ 1.  2.  3.  4.  5.  6.  9.  7.  8. 10. 13. nan 14. 12. 20. 15. 16. 11.]

num_req_bureau_day - [ 0. nan  1.  3.  2.  4.  5.  6.  9.]

num_req_bureau_week - [ 0. nan  1.  3.  2.  4.  5.  6.  8.  7.]

num_req_bureau_month - [ 0. nan  1.  2.  6.  3.  5.  7.  9.  4. 11.  8. 16. 12. 14. 10. 13. 17.
 24. 19. 15. 23. 18. 27.]

num_req_bureau_qrt - [  0.  nan   1.   2.   4.   3.   8.   5.   6.   7. 261.  19.]

num_req_bureau_year - [ 1.  0. nan  2.  4.  5.  3.  8.  6.  9.  7. 10. 11. 16. 12. 23. 15. 14.
 22. 17. 13. 21. 19.]

num_children - 0
num_family_members - 2
num_req_bureau_day - 33856
num_req_bureau_week - 33856
num_req_bureau_month - 33856
num_req_bureau_qrt - 33856
num_req_bureau_year - 33856
Num of Cols: 59


- Number of enquiries in a day before application gives few information about the risk assessment of the loan. So the column **num_rew_bureau_day** since the other similar columns have more importance

In [25]:
regex_past = 'past_loans.*'
df_columns[df_columns['New name'].str.match(regex_past)]

,New name,Description
54,past_loans_approved,Number of loans approved in the past for the client
55,past_loans_canceled,Number of loans canceled in the past for the client
56,past_loans_refused,Number of loans refused in the past for the client
57,past_loans_total,"Number of loans in the past for the client, regardless of the outcome"
58,past_loans_unused,Number of loans approved but not used in the past for the client


In [26]:
df_past_loans = df_loans_processed.filter(regex= (regex_past))
check_unique(df_past_loans)
check_nans(df_past_loans)

past_loans_approved - [ 1.  3.  5.  6.  4.  7.  2. nan  8. 12.  0.  9. 10. 13. 15. 11. 17. 16.
 14. 18. 19. 20. 24. 22. 21.]

past_loans_refused - [ 0.  3.  1. nan  6.  4.  2.  5. 10. 11.  8.  7.  9. 12. 21. 13. 14. 18.
 15. 16. 20. 17. 22. 19. 57. 25. 34. 24. 51. 23. 28. 40. 31. 38. 58. 26.
 30. 52.]

past_loans_canceled - [ 0.  1. nan 10.  8.  3.  2.  5. 12.  4.  9.  7.  6. 11. 20. 15. 13. 17.
 16. 19. 14. 32. 23. 30. 21. 25. 24. 18. 31. 37. 58. 22. 26. 28. 36. 27.
 33. 29. 47. 39. 68. 43. 35. 54. 51. 41.]

past_loans_unused - [ 0. nan  1.  3.  2.  4.  5.  6.  8.  7. 16. 10.]

past_loans_total - [ 1.  3.  9.  6.  5.  7.  4.  2. nan  8. 12. 22. 18. 10. 26. 11. 15. 14.
 17. 13. 23. 24. 27. 20. 16. 21. 19. 35. 29. 25. 31. 33. 28. 36. 41. 58.
 37. 40. 39. 30. 32. 34. 60. 50. 42. 43. 72. 44. 38. 52. 63. 55. 59. 73.
 49. 47. 61. 51. 54. 62. 46. 64. 53.]

past_loans_approved - 13669
past_loans_refused - 13669
past_loans_canceled - 13669
past_loans_unused - 13669
past_loans_total - 13669


In [27]:
cols_int = df_past_loans.columns.values.tolist() + df_num.columns.values.tolist() 
df_loans_processed.dropna(subset=cols_int, inplace=True)
print(f'Rows of raw data: {df_loans.shape[0]}\nRows of cleaned data: {df_loans_processed.shape[0]}')

Rows of raw data: 307511
Rows of cleaned data: 206416


#### Data Type Transformation

In [28]:
df_loans_processed = df_loans_processed.astype({col:'int64' for col in cols_int})
df_loans_processed[cols_int].head()

,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total,num_children,num_family_members,num_req_bureau_day,num_req_bureau_week,num_req_bureau_month,num_req_bureau_qrt,num_req_bureau_year
0,1,0,0,0,1,0,1,0,0,0,0,1
1,3,0,0,0,3,0,2,0,0,0,0,0
2,1,0,0,0,1,0,1,0,0,0,0,0
4,6,0,0,0,6,0,1,0,0,0,0,0
5,4,1,0,0,5,0,2,0,0,0,1,1


#### State of the data after processed

In [29]:
df_loans_processed.to_csv(DIR_DATA+'infringment_processed.csv')
df_loans_processed.shape

(206416, 59)